Takes as input a comma-separated file with the format: `line number, question part, question, answer`. Each line represents a separate sentence. The first line must be: `line, part, greek, english`

    Step 0: Imports

In [259]:
import stanza
import pandas as pd

from xml.dom import minidom
import os 

import xml.etree.ElementTree as gfg 

import lxml

!pip install pyconll
import pyconll

import glob
import csv


stanza.download('en')
stanza.download('grc')

nlp_grk = stanza.Pipeline('grc') 
nlp_en = stanza.Pipeline('en')


2022-07-18 14:37:30 INFO: Downloading default packages for language: en (English)...


2022-07-18 14:37:31 INFO: File exists: /Users/mallard/stanza_resources/en/default.zip
2022-07-18 14:37:41 INFO: Finished downloading models and saved to /Users/mallard/stanza_resources.


2022-07-18 14:37:41 INFO: Downloading default packages for language: grc (Ancient_Greek)...


2022-07-18 14:37:41 INFO: File exists: /Users/mallard/stanza_resources/grc/default.zip
2022-07-18 14:37:43 INFO: Finished downloading models and saved to /Users/mallard/stanza_resources.


2022-07-18 14:37:43 INFO: Loading these models for language: grc (Ancient_Greek):
| Processor | Package |
-----------------------
| tokenize  | proiel  |
| pos       | proiel  |
| lemma     | proiel  |
| depparse  | proiel  |

2022-07-18 14:37:43 INFO: Use device: cpu
2022-07-18 14:37:43 INFO: Loading: tokenize
2022-07-18 14:37:43 INFO: Loading: pos
2022-07-18 14:37:43 INFO: Loading: lemma
2022-07-18 14:37:43 INFO: Loading: depparse
2022-07-18 14:37:43 INFO: Done loading processors!


2022-07-18 14:37:45 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-07-18 14:37:45 INFO: Use device: cpu
2022-07-18 14:37:45 INFO: Loading: tokenize
2022-07-18 14:37:45 INFO: Loading: pos
2022-07-18 14:37:45 INFO: Loading: lemma
2022-07-18 14:37:45 INFO: Loading: depparse
2022-07-18 14:37:46 INFO: Loading: sentiment
2022-07-18 14:37:46 INFO: Loading: constituency
2022-07-18 14:37:47 INFO: Loading: ner
2022-07-18 14:37:47 INFO: Done loading processors!


    Step 1: Store contents of csv in dataframe

In [282]:
# def isValid(myList):
#     if myList[0][2] == '':
#         print("empty space found!")
#         return False
#     return True

In [301]:
# define the file names here:
file_list = glob.glob('lib/Crosby-Shaeffer-Lessons/*.csv')

file_lines = []

# loop through each file (lesson)
for x in file_list:
    # read the contents, save them as list
    with open(file_list[0], newline='') as f:
        reader = csv.reader(f, delimiter = ',')
        l = list(reader)
        if not l[0][0] == '':
            file_lines = file_lines + l
    
# convert the list to a dataframe
lines_df = pd.DataFrame(file_lines, columns=['line', 'part', 'greek', 'english'])

print(lines_df)


   line part                               greek  \
0     7    b                        ἦν στρατηγός   
1     7    b                       ἔχει ἀδελφούς   
2     7    b               παύουσι τὸν στρατηγόν   
3     7    b           οἱ στρατηγοὶ ἦσαν ἀδελφοί   
4     7    b                          ἦν ποταμός   
5     7    b    τῷ στρατηγῷ πέμπει τοὺς ἀδελφούς   
6     7    b               παύει τοὺς στρατηγούς   
7     7    b  πέμπουσι τὸν τοῦ στρατηγοῦ ἀδελφόν   
8     7    b                        ἦν στρατηγός   
9     7    b                       ἔχει ἀδελφούς   
10    7    b               παύουσι τὸν στρατηγόν   
11    7    b           οἱ στρατηγοὶ ἦσαν ἀδελφοί   
12    7    b                          ἦν ποταμός   
13    7    b    τῷ στρατηγῷ πέμπει τοὺς ἀδελφούς   
14    7    b               παύει τοὺς στρατηγούς   
15    7    b  πέμπουσι τὸν τοῦ στρατηγοῦ ἀδελφόν   

                                 english  
0                       He was a general  
1                        He h

    Step 2: Put dataframe info into lists

In [302]:
line_nums = lines_df['line'].tolist()
q_part = lines_df['part'].tolist()

greek_lines = lines_df['greek'].tolist()
english_lines = lines_df['english'].tolist()

    Step 3: Place the data on each word into a UD file format. Note that we can test this file with the CoNNL-U viewer.

Resources:

[UD file format](https://universaldependencies.org/format.html)

[CoNNL file viewer](https://universaldependencies.org/conllu_viewer.html)

In [304]:
treebank_file = 'UDtreebank.txt'
skip_lines = []
with open(treebank_file, 'w') as f:
      
    # loop through each line of text
    for index in range(0, len(english_lines)): 

        # get the data for each word
        grk_doc = nlp_grk(greek_lines[index])

        if len(grk_doc.sentences) > 0:
#             if len(grk_doc.sentences[0].words.text) < 0:
                
            for grk_token in grk_doc.sentences[0].words:
                # D: Word index, integer starting at 1 for each new sentence; may be a range for multiword tokens; may be a decimal number for empty nodes (decimal numbers can be lower than 1 but must be greater than 0).
                f.write(str(grk_token.id))
                f.write('\t')

                # FORM: Word form or punctuation symbol.
                f.write(grk_token.text)
                f.write('\t')

                # LEMMA: Lemma or stem of word form.
                f.write(grk_token.lemma)
                f.write('\t')

                # UPOS: Universal part-of-speech tag.
                f.write(grk_token.upos)
                f.write('\t')

                # XPOS: Language-specific part-of-speech tag; underscore if not available.
                f.write(grk_token.xpos)
                f.write('\t')

                # FEATS: List of morphological features from the universal feature inventory or from a defined language-specific extension; underscore if not available.
                f.write(str(grk_token.feats))
                f.write('\t')

                # HEAD: Head of the current word, which is either a value of ID or zero (0).
                f.write(str(grk_token.head))
                f.write('\t')

                # DEPREL: Universal dependency relation to the HEAD (root iff HEAD = 0) or a defined language-specific subtype of one.
                f.write(str(grk_token.deprel))
                f.write('\t')

                # DEPS: Enhanced dependency graph in the form of a list of head-deprel pairs.
                f.write(str(grk_token.deps))
                f.write('\t')

                # MISC: Any other annotation.
                # Sentence number/part
                f.write(str(line_nums[index]) + q_part[index])

                f.write('\n')

            f.write('\n')
        
        
        # get the data for each word
        en_doc = nlp_en(english_lines[index])

        if len(en_doc.sentences) > 0:
            for en_token in en_doc.sentences[0].words:
                # D: Word index, integer starting at 1 for each new sentence; may be a range for multiword tokens; may be a decimal number for empty nodes (decimal numbers can be lower than 1 but must be greater than 0).
                f.write(str(en_token.id))
                f.write('\t')

                # FORM: Word form or punctuation symbol.
                f.write(en_token.text)
                f.write('\t')

                # LEMMA: Lemma or stem of word form.
                f.write(en_token.lemma)
                f.write('\t')

                # UPOS: Universal part-of-speech tag.
                f.write(en_token.upos)
                f.write('\t')

                # XPOS: Language-specific part-of-speech tag; underscore if not available.
                f.write(en_token.xpos)
                f.write('\t')

                # FEATS: List of morphological features from the universal feature inventory or from a defined language-specific extension; underscore if not available.
                f.write(str(en_token.feats))
                f.write('\t')

                # HEAD: Head of the current word, which is either a value of ID or zero (0).
                f.write(str(en_token.head))
                f.write('\t')

                # DEPREL: Universal dependency relation to the HEAD (root iff HEAD = 0) or a defined language-specific subtype of one.
                f.write(str(en_token.deprel))
                f.write('\t')

                # DEPS: Enhanced dependency graph in the form of a list of head-deprel pairs.
                f.write(str(en_token.deps))
                f.write('\t')

                # MISC: Any other annotation.
                # Sentence number/part
                f.write(str(line_nums[index]) + q_part[index])

                f.write('\n')

            f.write('\n')


# Stanza Features (https://stanfordnlp.github.io/stanza/data_objects.html):

# "id"
# "text"
# "lemma"
# "upos"
# "xpos"
# "feats"
# "head"
# "deprel"
# "start_char"
# "end_char"
# "ner"


# UD Required Features:

# D: Word index, integer starting at 1 for each new sentence; may be a range for multiword tokens; may be a decimal number for empty nodes (decimal numbers can be lower than 1 but must be greater than 0).
# FORM: Word form or punctuation symbol.
# LEMMA: Lemma or stem of word form.
# UPOS: Universal part-of-speech tag.
# XPOS: Language-specific part-of-speech tag; underscore if not available.
# FEATS: List of morphological features from the universal feature inventory or from a defined language-specific extension; underscore if not available.
# HEAD: Head of the current word, which is either a value of ID or zero (0).
# DEPREL: Universal dependency relation to the HEAD (root iff HEAD = 0) or a defined language-specific subtype of one.
# DEPS: Enhanced dependency graph in the form of a list of head-deprel pairs.
# MISC: Any other annotation.

    Step 4: same as above, but make 2 files (one english, one greek) so we can compare them

In [305]:
treebank_file_en = 'english_UDtreebank.txt'
treebank_file_grk = 'greek_UDtreebank.txt'
skip_lines = []
with open(treebank_file_en, 'w') as f:
      
    # loop through each line of text
    for index in range(0, len(english_lines)): 
  
        # get the data for each word
        en_doc = nlp_en(english_lines[index])

        if len(en_doc.sentences) > 0:
            for en_token in en_doc.sentences[0].words:
                # D: Word index, integer starting at 1 for each new sentence; may be a range for multiword tokens; may be a decimal number for empty nodes (decimal numbers can be lower than 1 but must be greater than 0).
                f.write(str(en_token.id))
                f.write('\t')

                # FORM: Word form or punctuation symbol.
                f.write(en_token.text)
                f.write('\t')

                # LEMMA: Lemma or stem of word form.
                f.write(en_token.lemma)
                f.write('\t')

                # UPOS: Universal part-of-speech tag.
                f.write(en_token.upos)
                f.write('\t')

                # XPOS: Language-specific part-of-speech tag; underscore if not available.
                f.write(en_token.xpos)
                f.write('\t')

                # FEATS: List of morphological features from the universal feature inventory or from a defined language-specific extension; underscore if not available.
                f.write(str(en_token.feats))
                f.write('\t')

                # HEAD: Head of the current word, which is either a value of ID or zero (0).
                f.write(str(en_token.head))
                f.write('\t')

                # DEPREL: Universal dependency relation to the HEAD (root iff HEAD = 0) or a defined language-specific subtype of one.
                f.write(str(en_token.deprel))
                f.write('\t')

                # DEPS: Enhanced dependency graph in the form of a list of head-deprel pairs.
                f.write(str(en_token.deps))
                f.write('\t')

                # MISC: Any other annotation.
                # Sentence number/part
                f.write(str(line_nums[index]) + q_part[index])

                f.write('\n')

            f.write('\n')
            
            
with open(treebank_file_grk, 'w') as f:
      
    # loop through each line of text
    for index in range(0, len(english_lines)): 

        # get the data for each word
        grk_doc = nlp_grk(greek_lines[index])

        if len(grk_doc.sentences) > 0:
                
            for grk_token in grk_doc.sentences[0].words:
                # D: Word index, integer starting at 1 for each new sentence; may be a range for multiword tokens; may be a decimal number for empty nodes (decimal numbers can be lower than 1 but must be greater than 0).
                f.write(str(grk_token.id))
                f.write('\t')

                # FORM: Word form or punctuation symbol.
                f.write(grk_token.text)
                f.write('\t')

                # LEMMA: Lemma or stem of word form.
                f.write(grk_token.lemma)
                f.write('\t')

                # UPOS: Universal part-of-speech tag.
                f.write(grk_token.upos)
                f.write('\t')

                # XPOS: Language-specific part-of-speech tag; underscore if not available.
                f.write(grk_token.xpos)
                f.write('\t')

                # FEATS: List of morphological features from the universal feature inventory or from a defined language-specific extension; underscore if not available.
                f.write(str(grk_token.feats))
                f.write('\t')

                # HEAD: Head of the current word, which is either a value of ID or zero (0).
                f.write(str(grk_token.head))
                f.write('\t')

                # DEPREL: Universal dependency relation to the HEAD (root iff HEAD = 0) or a defined language-specific subtype of one.
                f.write(str(grk_token.deprel))
                f.write('\t')

                # DEPS: Enhanced dependency graph in the form of a list of head-deprel pairs.
                f.write(str(grk_token.deps))
                f.write('\t')

                # MISC: Any other annotation.
                # Sentence number/part
                f.write(str(line_nums[index]) + q_part[index])

                f.write('\n')

            f.write('\n')
        